# tdms原始数据读取 
## 不要随便运行， 注释掉了，要用再解除注释

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path("/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/High_similar_ML/data_process")
sys.path.insert(0, str(PROJECT_ROOT))

print("📌 Project root set to:", PROJECT_ROOT)


from my_lib.KAIST.KAIST_utils import process_kaist_file
from pathlib import Path

# =========================
# 1. 原始 TDMS 文件列表
# =========================
tdms_files = [
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFI_03.tdms",
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFI_10.tdms",
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFI_30.tdms",

    # "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_03.tdms",
    # "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_10.tdms",
    # "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_30.tdms",

    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Misalign_01.tdms",
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Misalign_03.tdms",
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Misalign_05.tdms",

    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Normal.tdms",

    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Unbalance_0583mg.tdms",
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Unbalance_1169mg.tdms",
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Unbalance_1751mg.tdms",
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Unbalance_2239mg.tdms",
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Unbalance_3318mg.tdms",
]


# =========================
# 2. 保存目录（Linux 侧）
# =========================
save_dir = Path("/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw")
save_dir.mkdir(parents=True, exist_ok=True)

# =========================
# 3. 批量处理
# =========================
saved_csv_files = []

for tdms_path in tdms_files:
    print(f"🚀 Processing: {tdms_path}")
    out_csv = process_kaist_file(
        file_path=tdms_path,
        save_dir=str(save_dir),
        data_type="temperature_current"
    )
    saved_csv_files.append(out_csv)
    print(f"✅ Saved to: {out_csv}")

print(f"\n🎉 Done! Total files processed: {len(saved_csv_files)}")



### 针对BPFO数据进行单独处理，这些数据应该是所有时间点都有缺失，所以用上面的逻辑提取出来是空文件

In [2]:
# ============================================================
# ⭐ 单 Cell：KAIST TDMS → CSV（current & temp）
#    - 自动识别 Log 组
#    - 第一列统一为 timestamp
#    - 所有通道对齐长度
#    - 空值填 null
#    - 不包含 label
# ============================================================

from nptdms import TdmsFile
import pandas as pd
import numpy as np
from pathlib import Path

# =========================
# 1️⃣ TDMS 文件列表
# =========================
tdms_files = [
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_03.tdms",
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_10.tdms",
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_30.tdms",
]

# =========================
# 2️⃣ 输出目录
# =========================
save_dir = Path("/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw")
save_dir.mkdir(parents=True, exist_ok=True)

# =========================
# 3️⃣ 转换函数
# =========================
def tdms_to_csv(tdms_path, save_dir, fs=None):
    """
    fs:
      - None  -> 使用 sample index 作为 timestamp
      - 数值  -> timestamp = index / fs
    """
    tdms = TdmsFile.read(tdms_path)
    log_group = tdms["Log"]

    channel_data = {}
    max_len = 0

    # 读取所有 channel
    for ch in log_group.channels():
        data = ch[:]
        name = ch.name.replace("/", "_")  # 列名安全化

        if len(data) > 0:
            channel_data[name] = data
            max_len = max(max_len, len(data))
            print(f"✅ {name}: {len(data)} samples")
        else:
            channel_data[name] = []
            print(f"⚠️ {name}: empty")

    # =========================
    # 生成 timestamp（第一列）
    # =========================
    if fs is None:
        # 用 sample index（最稳，不引入假物理意义）
        timestamp = np.arange(max_len)
    else:
        # 用真实采样率
        timestamp = np.arange(max_len) / fs

    aligned_data = {"timestamp": timestamp}

    # =========================
    # 对齐通道长度
    # =========================
    for name, data in channel_data.items():
        if len(data) == 0:
            aligned_data[name] = [np.nan] * max_len
        elif len(data) < max_len:
            aligned_data[name] = np.pad(
                data,
                (0, max_len - len(data)),
                constant_values=np.nan
            )
        else:
            aligned_data[name] = data

    df = pd.DataFrame(aligned_data)

    out_csv = save_dir / f"{Path(tdms_path).stem}.csv"
    df.to_csv(out_csv, index=False, na_rep="null")

    print(f"💾 Saved: {out_csv} | shape={df.shape}")
    return out_csv

# =========================
# 4️⃣ 批量执行
# =========================
saved_files = []

for tdms_path in tdms_files:
    print("=" * 60)
    print(f"🚀 Processing {tdms_path}")
    out = tdms_to_csv(tdms_path, save_dir, fs=None)  # fs=None → timestamp 为索引
    saved_files.append(out)

print("\n🎉 All done!")
for f in saved_files:
    print(" -", f)


🚀 Processing /home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_03.tdms
✅ cDAQ9185-1F486B5Mod1_ai0: 1536492 samples
✅ cDAQ9185-1F486B5Mod1_ai1: 1536492 samples
✅ cDAQ9185-1F486B5Mod2_ai0: 1536492 samples
⚠️ cDAQ9185-1F486B5Mod2_ai2: empty
⚠️ cDAQ9185-1F486B5Mod2_ai3: empty
💾 Saved: /home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw/0Nm_BPFO_03.csv | shape=(1536492, 6)
🚀 Processing /home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_10.tdms
✅ cDAQ9185-1F486B5Mod1_ai0: 1536492 samples
✅ cDAQ9185-1F486B5Mod1_ai1: 1536492 samples
✅ cDAQ9185-1F486B5Mod2_ai0: 1536492 samples
⚠️ cDAQ9185-1F486B5Mod2_ai2: empty
⚠️ cDAQ9185-1F486B5Mod2_ai3: empty
💾 Saved: /home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw/0Nm_BPF

## 重写列名

In [ ]:
from pathlib import Path
import pandas as pd

# =========================
# 1. 输入 / 输出目录
# =========================
src_dir = Path("/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw")
dst_dir = Path("/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw_withlabel")
dst_dir.mkdir(parents=True, exist_ok=True)

# =========================
# 2. 原始 → 标准列名映射
# =========================
column_rename_map = {
    "timestamp": "timestamp",

    "cDAQ9185-1F486B5Mod1/ai0": "Temp_A(°C)",
    "cDAQ9185-1F486B5Mod1/ai1": "Temp_B(°C)",

    "cDAQ9185-1F486B5Mod2/ai0": "I_U(A)",
    "cDAQ9185-1F486B5Mod2/ai2": "I_V(A)",
    "cDAQ9185-1F486B5Mod2/ai3": "I_W(A)",
}

raw_cols = set(column_rename_map.keys())
std_cols = set(column_rename_map.values())

# =========================
# 3. 批量处理
# =========================
csv_files = sorted(src_dir.glob("*.csv"))
print(f"📂 Found {len(csv_files)} CSV files")

for csv_path in csv_files:
    print(f"\n🔄 Processing: {csv_path.name}")

    df = pd.read_csv(csv_path)

    cols = set(df.columns)

    # =========================
    # ① 如果已经是标准列名 → 跳过
    # =========================
    if std_cols.issubset(cols):
        print("⏭️ 已是标准列名，跳过处理")
        out_path = dst_dir / csv_path.name
        df.to_csv(out_path, index=False)
        continue

    # =========================
    # ② 否则，检查是否具备原始列
    # =========================
    missing_raw = raw_cols - cols
    if missing_raw:
        raise ValueError(
            f"{csv_path.name} 缺少原始列，且不是标准格式: {missing_raw}"
        )

    # =========================
    # ③ 执行重命名
    # =========================
    df = df.rename(columns=column_rename_map)

    # 再次校验
    missing_after = std_cols - set(df.columns)
    if missing_after:
        raise ValueError(
            f"{csv_path.name} 重命名后缺少标准列: {missing_after}"
        )

    # =========================
    # ④ 保存
    # =========================
    out_path = dst_dir / csv_path.name
    df.to_csv(out_path, index=False)

    print(f"✅ 列名已统一，保存到: {out_path}")

print("\n🎉 所有文件处理完成。")


In [4]:
from pathlib import Path
import pandas as pd

# =========================
# 1️⃣ 指定这三个文件
# =========================
csv_files = [
    Path("/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw/0Nm_BPFO_03.csv"),
    Path("/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw/0Nm_BPFO_10.csv"),
    Path("/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw/0Nm_BPFO_30.csv"),
]

# =========================
# 2️⃣ 输出目录
# =========================
dst_dir = Path("/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw_withlabel")
dst_dir.mkdir(parents=True, exist_ok=True)

# =========================
# 3️⃣ 列名映射规则
# =========================
column_rename_map = {
    "timestamp": "timestamp",

    "cDAQ9185-1F486B5Mod1_ai0": "Temp_A(°C)",
    "cDAQ9185-1F486B5Mod1_ai1": "Temp_B(°C)",

    "cDAQ9185-1F486B5Mod2_ai0": "I_U(A)",
    "cDAQ9185-1F486B5Mod2_ai2": "I_V(A)",
    "cDAQ9185-1F486B5Mod2_ai3": "I_W(A)",
}

raw_cols = set(column_rename_map.keys())
std_cols = set(column_rename_map.values())

# =========================
# 4️⃣ 单独处理
# =========================
for csv_path in csv_files:
    print(f"\n🔄 Processing: {csv_path.name}")

    if not csv_path.exists():
        print("❌ 文件不存在，跳过")
        continue

    df = pd.read_csv(csv_path)
    cols = set(df.columns)

    # 已经是标准列名 → 直接拷贝
    if std_cols.issubset(cols):
        print("⏭️ 已是标准列名，直接保存")
        out_path = dst_dir / csv_path.name
        df.to_csv(out_path, index=False)
        continue

    # 检查原始列是否齐全
    missing_raw = raw_cols - cols
    if missing_raw:
        raise ValueError(
            f"{csv_path.name} 缺少原始列，且不是标准格式: {missing_raw}"
        )

    # 重命名
    df = df.rename(columns=column_rename_map)

    # 再校验
    missing_after = std_cols - set(df.columns)
    if missing_after:
        raise ValueError(
            f"{csv_path.name} 重命名后缺少标准列: {missing_after}"
        )

    out_path = dst_dir / csv_path.name
    df.to_csv(out_path, index=False)

    print(f"✅ 处理完成，保存到: {out_path}")

print("\n🎉 这三个文件已单独处理完成。")



🔄 Processing: 0Nm_BPFO_03.csv
✅ 处理完成，保存到: /home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw_withlabel/0Nm_BPFO_03.csv

🔄 Processing: 0Nm_BPFO_10.csv
✅ 处理完成，保存到: /home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw_withlabel/0Nm_BPFO_10.csv

🔄 Processing: 0Nm_BPFO_30.csv
✅ 处理完成，保存到: /home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw_withlabel/0Nm_BPFO_30.csv

🎉 这三个文件已单独处理完成。


## 加入标签

In [6]:
from pathlib import Path
import pandas as pd

# =========================
# 1. 数据目录
# =========================
data_dir = Path("/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw_withlabel")

# =========================
# 2. 文件名 → 标签 映射表
# =========================
label_map = {
    "0Nm_Normal": 1,

    "0Nm_BPFI_03": 2,
    "0Nm_BPFI_10": 3,
    "0Nm_BPFI_30": 4,

    "0Nm_BPFO_03": 5,
    "0Nm_BPFO_10": 6,
    "0Nm_BPFO_30": 7,

    "0Nm_Misalign_01": 8,
    "0Nm_Misalign_03": 9,
    "0Nm_Misalign_05": 10,

    "0Nm_Unbalance_0583mg": 11,
    "0Nm_Unbalance_1169mg": 12,
    "0Nm_Unbalance_1751mg": 13,
    "0Nm_Unbalance_2239mg": 14,
    "0Nm_Unbalance_3318mg": 15,
}

# =========================
# 3. 批量处理
# =========================
csv_files = sorted(data_dir.glob("*.csv"))
print(f"📂 Found {len(csv_files)} CSV files")

for csv_path in csv_files:
    stem = csv_path.stem
    print(f"\n🔄 Processing: {csv_path.name}")

    # ---------- ① 已有 label 列 → 跳过 ----------
    df = pd.read_csv(csv_path)

    if "label" in df.columns:
        print("⏭️ 已存在 label 列，跳过")
        continue

    # ---------- ② 文件名校验 ----------
    if stem not in label_map:
        raise ValueError(f"❌ 文件名未在映射表中: {csv_path.name}")

    label = label_map[stem]

    # ---------- ③ 添加 label ----------
    df["label"] = label

    # ---------- ④ 覆盖保存 ----------
    df.to_csv(csv_path, index=False)

    print(f"✅ Added label={label} to {csv_path.name}")

print("\n🎉 所有文件处理完成。")


📂 Found 15 CSV files

🔄 Processing: 0Nm_BPFI_03.csv
⏭️ 已存在 label 列，跳过

🔄 Processing: 0Nm_BPFI_10.csv
⏭️ 已存在 label 列，跳过

🔄 Processing: 0Nm_BPFI_30.csv
✅ Added label=4 to 0Nm_BPFI_30.csv

🔄 Processing: 0Nm_BPFO_03.csv
✅ Added label=5 to 0Nm_BPFO_03.csv

🔄 Processing: 0Nm_BPFO_10.csv
✅ Added label=6 to 0Nm_BPFO_10.csv

🔄 Processing: 0Nm_BPFO_30.csv
✅ Added label=7 to 0Nm_BPFO_30.csv

🔄 Processing: 0Nm_Misalign_01.csv
⏭️ 已存在 label 列，跳过

🔄 Processing: 0Nm_Misalign_03.csv
⏭️ 已存在 label 列，跳过

🔄 Processing: 0Nm_Misalign_05.csv
⏭️ 已存在 label 列，跳过

🔄 Processing: 0Nm_Normal.csv
⏭️ 已存在 label 列，跳过

🔄 Processing: 0Nm_Unbalance_0583mg.csv
⏭️ 已存在 label 列，跳过

🔄 Processing: 0Nm_Unbalance_1169mg.csv
⏭️ 已存在 label 列，跳过

🔄 Processing: 0Nm_Unbalance_1751mg.csv
⏭️ 已存在 label 列，跳过

🔄 Processing: 0Nm_Unbalance_2239mg.csv
⏭️ 已存在 label 列，跳过

🔄 Processing: 0Nm_Unbalance_3318mg.csv
⏭️ 已存在 label 列，跳过

🎉 所有文件处理完成。


## 提取每个样本5000个

In [7]:
import os
import glob
import pandas as pd

# ==========================
# 1️⃣ 输入 / 输出目录
# ==========================
src_dir = "/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw_withlabel"
dst_dir = "/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw_wl_5000"
os.makedirs(dst_dir, exist_ok=True)

output_csv = os.path.join(dst_dir, "0Nm_all_classes_merged_5000.csv")

samples_per_class = 5000
shuffle_each_class = True
random_state = 42

# ==========================
# 2️⃣ 读取 & 抽样
# ==========================
all_dfs = []

csv_files = sorted(
    f for f in glob.glob(os.path.join(src_dir, "*.csv"))
    if not f.endswith("_merged.csv")
)

assert len(csv_files) > 0, "❌ 输入目录下没有可用 CSV 文件"

print(f"🔍 共发现 {len(csv_files)} 个类别文件")

for csv_path in csv_files:
    class_name = os.path.basename(csv_path).replace(".csv", "")
    print(f"\n🔄 Processing: {class_name}")

    df = pd.read_csv(csv_path)

    # --------------------------
    # label 必须存在（你前一步已经统一过）
    # --------------------------
    if "label" not in df.columns:
        raise ValueError(f"{class_name} 缺少 label 列，请先检查上游步骤")

    # --------------------------
    # 抽样
    # --------------------------
    if len(df) >= samples_per_class:
        df_sampled = df.sample(
            n=samples_per_class,
            replace=False,
            random_state=random_state if shuffle_each_class else None
        )
    else:
        print(f"⚠️ {class_name} 样本不足 {samples_per_class}，仅有 {len(df)}，已全部使用")
        df_sampled = df

    df_sampled["source_file"] = class_name
    all_dfs.append(df_sampled)

    print(f"✅ {class_name}: {len(df_sampled)} samples")

# ==========================
# 3️⃣ 合并 & 保存
# ==========================
merged_df = pd.concat(all_dfs, ignore_index=True)

merged_df.to_csv(output_csv, index=False)

print("=" * 60)
print("🎉 合并完成")
print(f"📁 输出路径: {output_csv}")
print(f"📊 总样本数: {len(merged_df)}")
print("📈 各类别样本统计：")
print(merged_df["label"].value_counts())


🔍 共发现 15 个类别文件

🔄 Processing: 0Nm_BPFI_03
✅ 0Nm_BPFI_03: 5000 samples

🔄 Processing: 0Nm_BPFI_10
✅ 0Nm_BPFI_10: 5000 samples

🔄 Processing: 0Nm_BPFI_30
✅ 0Nm_BPFI_30: 5000 samples

🔄 Processing: 0Nm_BPFO_03
✅ 0Nm_BPFO_03: 5000 samples

🔄 Processing: 0Nm_BPFO_10
✅ 0Nm_BPFO_10: 5000 samples

🔄 Processing: 0Nm_BPFO_30
✅ 0Nm_BPFO_30: 5000 samples

🔄 Processing: 0Nm_Misalign_01
✅ 0Nm_Misalign_01: 5000 samples

🔄 Processing: 0Nm_Misalign_03
✅ 0Nm_Misalign_03: 5000 samples

🔄 Processing: 0Nm_Misalign_05
✅ 0Nm_Misalign_05: 5000 samples

🔄 Processing: 0Nm_Normal
✅ 0Nm_Normal: 5000 samples

🔄 Processing: 0Nm_Unbalance_0583mg
✅ 0Nm_Unbalance_0583mg: 5000 samples

🔄 Processing: 0Nm_Unbalance_1169mg
✅ 0Nm_Unbalance_1169mg: 5000 samples

🔄 Processing: 0Nm_Unbalance_1751mg
✅ 0Nm_Unbalance_1751mg: 5000 samples

🔄 Processing: 0Nm_Unbalance_2239mg
✅ 0Nm_Unbalance_2239mg: 5000 samples

🔄 Processing: 0Nm_Unbalance_3318mg
✅ 0Nm_Unbalance_3318mg: 5000 samples
🎉 合并完成
📁 输出路径: /home/charles/HZU/Data_processed/